+ 可视化<posetrack2json.py>生成的数据，检测标注信息是否正确

In [3]:
import cv2
import matplotlib.pyplot as plt 
from pathlib import Path 
import os 
import numpy as np 
from my_coco_tools import COCO
from tqdm import tqdm
import shutil

In [4]:
data_type = "val"   # train, val
ann_file = Path(f"posetrack/annotations/posetrack_{data_type}.json")
img_path = Path(f"posetrack/images/{data_type}")

PoseTrack的类别信息

In [5]:
categories = [
        {
            "supercategory": "person", 
            "id": 1, 
            "name": "person", 
            "keypoints": ['nose', 'head_bottom', 'head_top',
                        'left_ear', 'right_ear', 
                        'left_shoulder', 'right_shoulder',
                        'left_elbow', 'right_elbow',
                        'left_wrist', 'right_wrist',
                        'left_hip', 'right_hip',
                        'left_knee','right_knee', 
                        'left_ankle', 'right_ankle'], 
            "skeleton": [[16, 14], [14, 12], [17, 15],
                        [15, 13], [12, 13], [6, 12],
                        [7, 13], [6, 7], [6, 8], 
                        [7, 9], [8, 10], [9, 11],
                        [2, 3], [1, 2], [1, 3], 
                        [2, 4], [3, 5], [4, 6], [5, 7]]
        }
    ]
skeleton = np.array(categories[0]['skeleton']) - 1

In [6]:
def _int(v):
    return [int(vi) for vi in v]

def draw_pose(img, kpts, bbox, bbox_head, thickness=2, color=(0, 255, 0), skeleton=None):
    """
        img: cv2.imread
        kpt: ann_info['keypoints']
        bbox:ann_info['bbox']
        kpt_type: 'coco' or 'mixed', default to 'coco'
    """
    kpts = np.array(kpts).reshape((-1, 3))
    # 1 keypoints
    for x, y, vis in kpts:
        if vis > 0:
            cv2.circle(img, (int(x), int(y)), thickness+2, color, cv2.FILLED)

    # 2 bbox
    left_top = bbox[:2]
    right_bottom = bbox[0] + bbox[2], bbox[1] + bbox[3]
    cv2.rectangle(img, _int(left_top), _int(right_bottom), (0, 0, 255), thickness=thickness)  

    # 2 bbox_head
    left_top = bbox_head[:2]
    right_bottom = bbox_head[0] + bbox_head[2], bbox_head[1] + bbox_head[3]
    cv2.rectangle(img, _int(left_top), _int(right_bottom), (255, 0, 255), thickness=thickness)  

    if skeleton is not None:
        for pairs in skeleton:
            if kpts[pairs[0], 2] > 0 and kpts[pairs[1], 2] > 0:
                pt1 = kpts[pairs[0], :2].astype(np.int32).tolist()
                pt2 = kpts[pairs[1], :2].astype(np.int32).tolist()
                cv2.line(img, pt1, pt2, color, thickness)
    return img


In [7]:
def show_img(file, img_dir):
    print(f"Press 'q' to quit!")
    dataset = COCO(str(file))
    imgIds = dataset.getImgIds()
    cv2.namedWindow("images")
    for img_id in tqdm(imgIds, desc=f"{file.stem}"):
        img_info = dataset.loadImgs(img_id)[0]
        img_file = img_dir.joinpath(img_info['file_name'])

        if img_file.exists():
            img = plt.imread(str(img_file))
            annIds = dataset.getAnnIds(imgIds=img_id)
            if len(annIds) == 0:
                # raise ValueError(f"{len(annIds)=}")
                cv2.imshow("images", img)
                key = cv2.waitKey()
                if key & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    return 
                continue
            
            for ann_id in annIds:
                ann_info = dataset.loadAnns(ann_id)[0]
                kpts = ann_info.get('keypoints', False)
                bbox = ann_info.get('bbox', False)
                bbox_head = ann_info.get('bbox_head', False)
                if kpts and bbox and bbox_head:
                    img = draw_pose(img, kpts, bbox, bbox_head, skeleton=skeleton)

            cv2.imshow("images", img)
            key = cv2.waitKey(2)
            if key & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                return 
        else:
            print(f"img_file not exists!")
    cv2.destroyAllWindows()


In [8]:
show_img(file=ann_file, img_dir=img_path)

Press 'q' to quit!


posetrack_train:  20%|█▉        | 3529/18025 [02:51<11:43, 20.61it/s]
